# 训练数据nan检查

In [ ]:
nan_id = '518880_1699323787'
train_folder = r'D:\L2_DATA_T0_ETF\train_data\lable_1_combine_data'

from dl_helper.models.binctabl import m_bin_ctabl
t1, t2, t3, t4 = [100, 30, 10, 1]
d1, d2, d3, d4 = [20, 20, 10, 3]
model = m_bin_ctabl(d2, d1, t1, t2, d3, t3, d4, t4)
model

m_bin_ctabl(
  (BiN): BiN()
  (BL): BL_layer(
    (activation): ReLU()
  )
  (BL2): BL_layer(
    (activation): ReLU()
  )
  (TABL): TABL_layer(
    (activation): ReLU()
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
import pickle
import os, torch, datetime

import pandas as pd
import numpy as np

In [ ]:
nan_date =  datetime.datetime.fromtimestamp(int(nan_id.split('_')[1])).strftime('%Y%m%d')
print(f'nan_date: {nan_date}')
nan_file = f'{nan_date}.pkl'

# 递归 train_folder ，获得 20231107.pkl 文件路径
train_file = ''
for root, dirs, files in os.walk(train_folder):
    if nan_file in files:
        train_file = os.path.join(root, nan_file)
        break

print(f'nan_file: {train_file}')

# 读取文件
_ids,_mean_std, _x, _y, _raw = pickle.load(open(train_file, 'rb'))
print(f'样本数量: {len(_ids)}')

# 检查 _raw 是否包含 NaN 或 Inf
nan_indices = _raw.index[_raw.isna().any(axis=1)].tolist()
inf_indices = _raw.index[np.isinf(_raw.values).any(axis=1)].tolist()
if nan_indices:
    print("raw文件包含NaN值的行:\n", _raw.iloc[nan_indices])
if inf_indices:
    print("raw文件包含Inf值的行:\n", _raw.iloc[inf_indices])

idx = _ids.index(nan_id)
print(f'idx: {idx}')

# x, y, mm = batch
a, b = _x[idx]
x = torch.from_numpy(_raw.iloc[a:b, :].values).unsqueeze(0).float()
y = torch.tensor(_y[idx]).unsqueeze(0).float()
mm = torch.tensor(_mean_std[idx]).unsqueeze(0).float()

print(x.shape, y.shape, mm.shape)

nan_date: 20231107
nan_file: D:\L2_DATA_T0_ETF\train_data\lable_1_combine_data\train\20231107.pkl
样本数量: 8007
idx: 5129
torch.Size([1, 105, 153]) torch.Size([1, 28]) torch.Size([1, 151, 2])


In [ ]:
torch.isnan(x).any(), torch.isinf(x).any()

(tensor(False), tensor(False))

In [ ]:
# 数据预处理的代码
x = x[:, -100:, :20]
x = torch.transpose(x, 1, 2)

# 归一化
x -= mm[:, :20, :1]
x /= (mm[:, :20, 1:] -  mm[:, :20, :1])

In [ ]:
out = model(x)

In [ ]:
torch.isnan(out).any(), torch.isinf(out).any()

(tensor(True), tensor(False))